In [1]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
pd.set_option('display.max_rows', None)

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [2]:
output_path = "../data/processed/merged_df.csv"
merged_df = pd.read_csv(output_path)

# this is temporal, look for a way to fill this
merged_df = merged_df[merged_df['Country'] != 'Taiwan']

# Create Train & Test 
np.random.seed(42)
merged_df['is_train'] = np.random.choice([1, 0], size=len(merged_df), p=[0.8, 0.2])

In [3]:

def calculate_metrics(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "R²": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAPE": np.mean(np.abs((y_true - y_pred) / y_true)) * 100,
        "WAPE": np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100
    }

def train_and_evaluate(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return calculate_metrics(y_test, y_pred)

def add_metadata(metrics, region, target, strategy, features_used):
    metrics.update({
        'Region': region,
        'Target': target,
        'Strategy': strategy,
        'Model': 'Linear Regression',
        'Features Used': ', '.join(features_used)
    })
    return metrics

def reorder_columns(df):
    column_order = [
        'Region', 'Target', 'Strategy', 'Model', 'Features Used', 'Features Abbreviated',
        'MSE', 'R²', 'MAE', 'RMSE', 'MAPE', 'WAPE'
    ]
    existing_columns = df.columns.tolist()
    final_columns = [col for col in column_order if col in existing_columns]
    return df[final_columns]


def within_domain(df, features, target_columns):
    results = []
    for region in df['Region Grouped'].unique():
        region_data = df[df['Region Grouped'] == region].copy()
        mask = np.random.rand(len(region_data)) < 0.8
        
        for target in target_columns:
            X_train = region_data[features][mask]
            X_test = region_data[features][~mask]
            y_train = region_data[target][mask]
            y_test = region_data[target][~mask]
            
            metrics = train_and_evaluate(X_train, X_test, y_train, y_test)
            results.append(add_metadata(metrics, region, target, 'Within-Domain', features))
    return reorder_columns(pd.DataFrame(results).round(2))

def cross_domain(df, features, target_columns):
    results = []
    regions = df['Region Grouped'].unique()
    
    for test_region in regions:
        train_data = df[df['Region Grouped'] != test_region]
        test_data = df[df['Region Grouped'] == test_region]
        
        for target in target_columns:
            X_train = train_data[features]
            X_test = test_data[features]
            y_train = train_data[target]
            y_test = test_data[target]
            
            metrics = train_and_evaluate(X_train, X_test, y_train, y_test)
            results.append(add_metadata(metrics, test_region, target, 'Cross-Domain', features))
    return reorder_columns(pd.DataFrame(results).round(2))

def domain_adaptation(df, features, target_columns):
    results = []
    regions = df['Region Grouped'].unique()
    
    train_mask = np.random.rand(len(df)) < 0.8
    X_train_all = df[features][train_mask]
    
    for region in regions:
        test_data = df[(df['Region Grouped'] == region) & (~train_mask)]
        
        for target in target_columns:
            y_train = df[target][train_mask]
            y_test = test_data[target]
            X_test = test_data[features]
            
            metrics = train_and_evaluate(X_train_all, X_test, y_train, y_test)
            results.append(add_metadata(metrics, region, target, 'Domain-Adaptation', features))
    return reorder_columns(pd.DataFrame(results).round(2))

def add_abbreviated_feature_column(df, features, feature_abbreviations):
    abbreviated_features = [feature_abbreviations.get(feature, feature) for feature in features]
    df['Features Abbreviated'] = ' | '.join(abbreviated_features)
    return df

def append_total_row(results_df):
    total_row_df = pd.DataFrame()
    
    for target in results_df['Target'].unique():
        target_df = results_df[results_df['Target'] == target]
        metrics_avg = target_df[['MSE', 'R²', 'MAE', 'RMSE', 'MAPE', 'WAPE']].mean()
        
        total_row = {
            'Region': 'Total',
            'Target': target,
            'Strategy': results_df['Strategy'].iloc[0],
            'Model': results_df['Model'].iloc[0],
            'Features Used': results_df['Features Used'].iloc[0],
            'Features Abbreviated': results_df['Features Abbreviated'].iloc[0] if 'Features Abbreviated' in results_df.columns else 'N/A',
            'MSE': metrics_avg['MSE'],
            'R²': metrics_avg['R²'],
            'MAE': metrics_avg['MAE'],
            'RMSE': metrics_avg['RMSE'],
            'MAPE': metrics_avg['MAPE'],
            'WAPE': metrics_avg['WAPE']
        }
        
        total_row_df = pd.concat([total_row_df, pd.DataFrame([total_row])], ignore_index=True)
    
    return reorder_columns(pd.concat([results_df, total_row_df], ignore_index=True))


def run_all_strategies(df, features, target_columns, seed=42):
    np.random.seed(seed)
    
    within_domain_results = within_domain(df, features, target_columns)
    cross_domain_results = cross_domain(df, features, target_columns)
    domain_adaptation_results = domain_adaptation(df, features, target_columns)
    return within_domain_results, cross_domain_results, domain_adaptation_results

def filter_and_concatenate(*dfs):
    filtered_dfs = [df[df['Region'] == 'Total'] for df in dfs]
    concatenated_df = pd.concat(filtered_dfs, ignore_index=True)
    return concatenated_df

def filter_and_concatenate(*dfs):
    filtered_dfs = [df[df['Region'] == 'Total'] for df in dfs]
    concatenated_df = pd.concat(filtered_dfs, ignore_index=True)
    return concatenated_df


In [4]:
features = [
    "nearest_hdd", "GDP_2022", "Urbanization_Rate_2022",
    "latitude", "longitude", "Population_2023", "Paris_Agreement"
]

feature_abbreviations = {
    "nearest_hdd": "HDD",
    "GDP_2022": "GDP",
    "Urbanization_Rate_2022": "URB",
    "latitude": "Lat",
    "longitude": "Long",
    "Population_2023": "Pop",
    "Paris_Agreement": "Paris"
}

target_columns = [
    "Residential EUI (kWh/m2/year)",
    "Non-residential EUI (kWh/m2/year)"
]

within_domain_results, cross_domain_results, domain_adaptation_results = run_all_strategies(
    df=merged_df,
    features=features,
    target_columns=target_columns,
    seed=42
)

within_domain_results = add_abbreviated_feature_column(within_domain_results, features, feature_abbreviations)
cross_domain_results = add_abbreviated_feature_column(cross_domain_results, features, feature_abbreviations)
domain_adaptation_results = add_abbreviated_feature_column(domain_adaptation_results, features, feature_abbreviations)

within_domain_results = append_total_row(within_domain_results)
cross_domain_results = append_total_row(cross_domain_results)
domain_adaptation_results = append_total_row(domain_adaptation_results)

concatenated_results = filter_and_concatenate(
    within_domain_results,
    cross_domain_results,
    domain_adaptation_results
)

concatenated_results

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Total,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,929.068,0.386,20.400,25.408,15.894,14.604
1,Total,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1457.406,-0.892,24.172,31.774,14.468,12.486
2,Total,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2123.310,-0.416,33.394,41.186,33.776,27.508
3,Total,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,9672.768,-15.800,70.330,86.898,49.086,46.574
4,Total,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1518.924,-0.330,27.084,35.554,21.252,20.722
5,Total,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2864.274,-13.212,37.034,45.182,21.908,20.282


In [5]:
# SAVE RESULTS 

results_dir = '../results/'
os.makedirs(results_dir, exist_ok=True)

date_str = datetime.now().strftime("%Y%m%d_%H%M")

within_domain_results.to_csv(
    os.path.join(results_dir, f'results_{date_str}_within_domain.csv'), 
    index=False
)
cross_domain_results.to_csv(
    os.path.join(results_dir, f'results_{date_str}_cross_domain.csv'), 
    index=False
)
domain_adaptation_results.to_csv(
    os.path.join(results_dir, f'results_{date_str}_domain_adaptation.csv'), 
    index=False
)

concatenated_results.to_csv(  
    os.path.join(results_dir, f'results_{date_str}_Total.csv'), 
    index=False )

In [6]:
concatenated_results.sort_values(by=['Target', 'Strategy'], ascending=False).reset_index(drop=True)

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Total,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,929.068,0.386,20.400,25.408,15.894,14.604
1,Total,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1518.924,-0.330,27.084,35.554,21.252,20.722
2,Total,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2123.310,-0.416,33.394,41.186,33.776,27.508
3,Total,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1457.406,-0.892,24.172,31.774,14.468,12.486
4,Total,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2864.274,-13.212,37.034,45.182,21.908,20.282
5,Total,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,9672.768,-15.800,70.330,86.898,49.086,46.574


In [7]:
within_domain_results

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Asia & Oceania,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,833.790,0.730,18.670,28.880,18.750,18.700
1,Asia & Oceania,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,596.400,0.710,17.360,24.420,12.250,12.120
2,Europe,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2882.020,0.380,43.640,53.680,27.010,21.260
3,Europe,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,3930.520,0.120,47.930,62.690,27.310,22.040
4,Africa,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,26.830,0.230,4.200,5.180,6.210,6.080
5,Africa,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,174.870,0.020,4.560,13.220,8.600,4.470
6,Central and South America,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,134.040,0.450,9.770,11.580,11.930,12.020
7,Central and South America,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,70.400,-0.280,6.850,8.390,6.780,6.550
8,Northern America,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,768.660,0.140,25.720,27.720,15.570,14.960
9,Northern America,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2514.840,-5.030,44.160,50.150,17.400,17.250


In [8]:
cross_domain_results

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Asia & Oceania,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,3507.620,-0.050,46.100,59.230,73.580,50.210
1,Asia & Oceania,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,28985.180,-15.400,129.150,170.250,106.960,96.900
2,Europe,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,4843.730,0.160,54.660,69.600,34.770,26.030
3,Europe,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,5997.010,0.200,52.900,77.440,25.410,23.200
4,Africa,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,370.220,-1.400,14.780,19.240,19.780,20.690
5,Africa,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2170.150,-30.980,40.930,46.580,38.980,39.000
6,Central and South America,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,339.400,-0.170,15.190,18.420,19.080,18.600
7,Central and South America,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1921.000,-26.600,39.440,43.830,38.740,37.620
8,Northern America,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1555.580,-0.620,36.240,39.440,21.670,22.010
9,Northern America,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,9290.500,-6.220,89.230,96.390,35.340,36.150


In [9]:
domain_adaptation_results

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Asia & Oceania,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2583.230,0.290,25.570,50.830,25.560,27.530
1,Asia & Oceania,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1153.400,-0.430,23.900,33.960,19.460,18.810
2,Europe,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2960.800,0.070,47.790,54.410,27.180,21.500
3,Europe,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,2826.470,-0.000,34.800,53.160,23.600,16.500
4,Africa,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,397.430,-1.650,13.340,19.940,16.970,18.450
5,Africa,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,349.750,-36.990,14.270,18.700,13.470,13.570
6,Central and South America,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,215.280,-0.130,12.100,14.670,16.250,15.700
7,Central and South America,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,518.140,-7.300,18.290,22.760,18.290,17.540
8,Northern America,Residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,1437.880,-0.230,36.620,37.920,20.300,20.430
9,Northern America,Non-residential EUI (kWh/m2/year),Domain-Adaptation,Linear Regression,"nearest_hdd, GDP_2022, Urbanization_Rate_2022,...",HDD | GDP | URB | Lat | Long | Pop | Paris,9473.610,-21.340,93.910,97.330,34.720,34.990
